# 测试AXI-Lite Adder IP

## 载入Overlay

In [20]:
from pynq import Overlay, allocate

overlay = Overlay("./adder.bit")

通过`ip_dict`属性可以查看设计中各IP的基本信息：

In [21]:
overlay.ip_dict

{'axilite_adder_0': {'fullpath': 'axilite_adder_0',
  'type': 'amd.com:user:axilite_adder:1.0',
  'bdtype': None,
  'state': None,
  'addr_range': 65536,
  'phys_addr': 1136656384,
  'mem_id': 'S00_AXI',
  'memtype': 'REGISTER',
  'gpio': {},
  'interrupts': {},
  'parameters': {'C_S00_AXI_DATA_WIDTH': '32',
   'C_S00_AXI_ADDR_WIDTH': '4',
   'Component_Name': 'design_1_axilite_adder_0_0',
   'EDK_IPTYPE': 'PERIPHERAL',
   'C_S00_AXI_BASEADDR': '0x43C00000',
   'C_S00_AXI_HIGHADDR': '0x43C0FFFF'},
  'registers': {},
  'device': <pynq.pl_server.embedded_device.EmbeddedDevice at 0xb34d51d8>,
  'driver': pynq.overlay.DefaultIP},
 'processing_system7_0': {'gpio': {},
  'interrupts': {},
  'parameters': {'C_EN_EMIO_PJTAG': '0',
   'C_EN_EMIO_ENET0': '0',
   'C_EN_EMIO_ENET1': '0',
   'C_EN_EMIO_TRACE': '0',
   'C_INCLUDE_TRACE_BUFFER': '0',
   'C_TRACE_BUFFER_FIFO_SIZE': '128',
   'USE_TRACE_DATA_EDGE_DETECTOR': '0',
   'C_TRACE_PIPELINE_WIDTH': '8',
   'C_TRACE_BUFFER_CLOCK_DELAY': '12',
 

## 使用Overlay属性进行读写

提取出我们设计的AXI-Lite加法器的IP

In [24]:
axilite = overlay.axilite_adder_0

直接根据地址Offset写入两个随机数，并读取0xc来获得计算结果

In [25]:
import random
input_1 = int(random.random()*1000)
input_2 = int(random.random()*1000)
gt = input_1 + input_2

axilite.write(0x0, input_1)
axilite.write(0x4, input_2)
res = axilite.read(0xc)

print("{}+{}={}".format(input_1, input_2, res))

68+614=682


### 使用MMIO进行读写

在使用MMIO进行读写时，我们需要指定Base Address，即之前Vivado中所看的IP的Master Base Address(在上方的overlay.ip_dict输出中也可以看到)，为0x43C0_0000，范围为64K

In [26]:
IP_BASE_ADDRESS = 0x43C0_0000
ADDRESS_RANGE = 0xFFFF

from pynq import MMIO
mmio = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)

在Base Address的基础上添加指定Offset、写入两个随机数，同样读取0xC上的结果

In [28]:
input_1 = int(random.random()*1000)
input_2 = int(random.random()*1000)

ADDRESS_OFFSET = 0x0
mmio.write(ADDRESS_OFFSET, input_1)

ADDRESS_OFFSET = 0x4
mmio.write(ADDRESS_OFFSET, input_2)

ADDRESS_OFFSET = 0xc
res = mmio.read(ADDRESS_OFFSET)

print("{}+{}={}".format(input_1, input_2, res))

643+670=1313
